In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery

In [3]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [4]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9991.000000,10000.000000,10000.000000,9881.000000
mean,7.279259,27.332300,1.033200,38.704888
std,1.317603,6.238122,0.188948,2.602659
min,0.661387,12.000000,1.000000,19.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.808857,53.000000,3.000000,47.000000


In [5]:
df['is_male'].value_counts()

True     5158
False    4842
Name: is_male, dtype: int64

In [6]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [7]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])

In [8]:
data['is_male'] = data['is_male'].astype(int)

In [10]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
757,1,23,1,39.0
6544,1,19,1,38.0
3190,1,28,1,40.0
2064,0,32,1,40.0
4066,0,17,1,40.0


In [11]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [12]:
model = xgb.XGBRegressor(objective='reg:squarederror')

In [13]:
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
y_pred = model.predict(x_test)

for i in range(20):
    print('Predicted weight: ', y_pred[i])
    print('Actual weight: ', y_test.iloc[i])
    print()

Save model as a local instance

In [16]:
model.save_model('model.bst')

In [17]:
!gcloud config list project --format "value(core.project)"

kens-maps-1548181149530


In [18]:
GCP_PROJECT = 'kens-maps-1548181149530'
MODEL_BUCKET = 'gs://kens-maps-1548181149530'
VERSION_NAME = 'v1'
MODEL_NAME = 'baby_weight'

In [19]:
!gsutil mb $MODEL_BUCKET

Creating gs://kens-maps-1548181149530/...


In [20]:
!gsutil cp ./model.bst $MODEL_BUCKET

Copying file://./model.bst [Content-Type=application/octet-stream]...
/ [1 files][318.6 KiB/318.6 KiB]                                                
Operation completed over 1 objects/318.6 KiB.                                    


In [21]:
!gcloud ai-platform models create $MODEL_NAME --region=us-central1

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/kens-maps-1548181149530/models/baby_weight].


In [22]:
!gcloud ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--framework='XGBOOST' \
--runtime-version=2.5 \
--origin=$MODEL_BUCKET \
--python-version=3.7 \
--project=$GCP_PROJECT \
--region=us-central1

Using endpoint [https://us-central1-ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [23]:
%%writefile predictions.json
[0.0, 33.0, 1.0, 27.0]
[1.0, 26.0, 1.0, 40.0]

Writing predictions.json


In [24]:
prediction = !gcloud ai-platform predict --model=$MODEL_NAME --json-instances=predictions.json --version=$VERSION_NAME
print(prediction.s)

Using endpoint [https://us-central1-ml.googleapis.com/] [2.4736833572387695, 7.705554485321045]
